# Episode 16 — Saving and Loading Circuits with QPY Serialization

QPY is Qiskit's **binary, forward‑compatible** format for serializing `QuantumCircuit` objects. 
Use it when you want to persist full circuit structure (including metadata) or move circuits between machines and Python versions.

In this notebook you will:
1. Save and load a circuit with `qiskit.qpy.dump` / `load`.
2. Serialize **multiple** circuits in a single file.
3. Use **gzip** compression and **version** checks.
4. Verify **round‑trip integrity** (matrix equality).
5. Complete a small **challenge**.

> Tip: QPY is forward compatible (newer Qiskit can read older files). Backwards compatibility (old Qiskit reading newer QPY) is **not** guaranteed.

## 0. Imports and helpers

In [1]:
from __future__ import annotations
import os, gzip, json, tempfile, contextlib
from pathlib import Path
from typing import Iterable, List

import numpy as np
from qiskit import QuantumCircuit, qpy
from qiskit.quantum_info import Operator

def save_qpy(path: str | os.PathLike, circuits):
    """Save a circuit or list of circuits to QPY."""
    with open(path, "wb") as f:
        qpy.dump(circuits, f)

def load_qpy(path: str | os.PathLike) -> List[QuantumCircuit]:
    """Load QPY file and return a list of circuits."""
    with open(path, "rb") as f:
        return qpy.load(f)

@contextlib.contextmanager
def tempdir():
    d = tempfile.TemporaryDirectory()
    try:
        yield Path(d.name)
    finally:
        d.cleanup()

## 1. Save and load a single circuit
We’ll create a tiny Bell circuit, dump it to a `.qpy` file, and load it back.

In [2]:
with tempdir() as d:
    bell = QuantumCircuit(2, name="Bell", metadata={"author": "you"})
    bell.h(0)
    bell.cx(0, 1)
    bell.measure_all()

    qpy_path = d / "bell.qpy"
    save_qpy(qpy_path, bell)

    loaded = load_qpy(qpy_path)
    print(f"Loaded {len(loaded)} circuit(s)")
    loaded[0].draw("mpl")

Loaded 1 circuit(s)


## 2. Store multiple circuits in one file
QPY can hold a **list** of circuits. Loading always returns a list—even if you stored just one.

In [3]:
with tempdir() as d:
    c1 = QuantumCircuit(1, name="x_gate"); c1.x(0)
    c2 = QuantumCircuit(1, name="h_gate"); c2.h(0)
    bundle_path = d / "bundle.qpy"
    save_qpy(bundle_path, [c1, c2])
    back = load_qpy(bundle_path)
    print([qc.name for qc in back])

['x_gate', 'h_gate']


## 3. Optional: gzip‑compressed QPY
You can compress the payload using `gzip` to reduce disk usage (nice for large collections).

In [4]:
with tempdir() as d:
    qc = QuantumCircuit(3, name="ghz")
    qc.h(0); qc.cx(0,1); qc.cx(1,2)

    gz_path = d / "ghz.qpy.gz"
    with gzip.open(gz_path, "wb") as f:
        qpy.dump(qc, f)

    with gzip.open(gz_path, "rb") as f:
        restored = qpy.load(f)

    print(restored[0].name)
    restored[0].draw("mpl")

ghz


## 4. Version checks and round‑trip integrity
You can inspect a file's QPY **format version** with `qpy.get_qpy_version`. 
We'll also verify that a circuit's action is preserved by comparing unitaries before and after serialization.

In [5]:
with tempdir() as d:
    qc = QuantumCircuit(2, name="unitary_test")
    qc.ry(0.3, 0); qc.rz(0.7, 1); qc.cx(0,1)
    U_before = Operator(qc)

    p = d / "u.qpy"
    save_qpy(p, qc)
    ver = qpy.get_qpy_version(open(p, "rb"))
    print("QPY format version:", ver)

    qc2 = load_qpy(p)[0]
    U_after = Operator(qc2)
    print("Unitary preserved:", np.allclose(U_before.data, U_after.data))

QPY format version: 15
Unitary preserved: True


### Notes & Pitfalls
- QPY is **forward compatible**. Loading with **older** Qiskit versions may fail.
- Circuit **metadata** is serialized using JSON. Custom objects need custom serializers.
- If you intentionally need to target an **older QPY format**, `qpy.dump(..., version=...)` lets you select (within supported bounds). If a feature cannot be represented in that version, an `UnsupportedFeatureForVersion` error is raised.
- Pulse payloads from legacy versions deserialize as undefined custom instructions (no calibrations) in Qiskit ≥ 2.0.

## 5. Mini Challenge
**Task:**
1. Build a 2‑qubit circuit with a parameter (e.g., `theta`).
2. Save two files: one with the default QPY version and one with an explicitly lower `version` (if supported in your install).
3. Load both files and show that drawing the circuits yields the same structure.
4. Confirm that their **operators** are identical when substituting a numeric value for the parameter.

<details>
<summary>Show solution</summary>

```python
from qiskit.circuit import Parameter
from qiskit import qpy
from qiskit.quantum_info import Operator
import numpy as np

theta = Parameter('θ')
qc = QuantumCircuit(2, name='param_demo')
qc.rx(theta, 0)
qc.ry(0.25, 1)
qc.cx(0, 1)

with tempdir() as d:
    p_def = d / 'param_default.qpy'
    with open(p_def, 'wb') as f:
        qpy.dump(qc, f)  # default (latest) version

    # If your installed Qiskit supports exporting an older version, set it here.
    # Fallback: write again with current version if not supported.
    p_old = d / 'param_old.qpy'
    try:
        with open(p_old, 'wb') as f:
            qpy.dump(qc, f, version=qpy.QPY_COMPATIBILITY_VERSION)
    except ValueError:
        with open(p_old, 'wb') as f:
            qpy.dump(qc, f)  # same as default when older not allowed

    q1 = qpy.load(open(p_def, 'rb'))[0]
    q2 = qpy.load(open(p_old, 'rb'))[0]
    display(q1.draw('mpl'))
    display(q2.draw('mpl'))

    U1 = Operator(q1.assign_parameters({theta: 0.42}))
    U2 = Operator(q2.assign_parameters({theta: 0.42}))
    print('Same unitary after assign:', np.allclose(U1.data, U2.data))
```

</details>

## Additional information

**Created by:** Ricard Santiago Raigada García

**Version:** 1.0.0